In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.io import arff
import pickle

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

For the diabetes dataset https://www.hindawi.com/journals/bmri/2014/781670/tab1/

In [ ]:
#dictionaries with key: dataset, value: list of lists, 0: continuos variables, 1: discrete variables
columns_type_dataset = {'yeast':[['Att1', 'Att2', 'Att3', 'Att4', 'Att5', 'Att6', 'Att7', 'Att8', 'Att9', 'Att10', 'Att11', 'Att12', 'Att13', 'Att14', 'Att15', 'Att16', 'Att17', 'Att18', 'Att19', 'Att20', 'Att21', 'Att22', 'Att23', 'Att24', 'Att25', 'Att26', 'Att27', 'Att28', 'Att29', 'Att30', 'Att31', 'Att32', 'Att33', 'Att34', 'Att35', 'Att36', 'Att37', 'Att38', 'Att39', 'Att40', 'Att41', 'Att42', 'Att43', 'Att44', 'Att45', 'Att46', 'Att47', 'Att48', 'Att49', 'Att50', 'Att51', 'Att52', 'Att53', 'Att54', 'Att55', 'Att56', 'Att57', 'Att58', 'Att59', 'Att60', 'Att61', 'Att62', 'Att63', 'Att64', 'Att65', 'Att66', 'Att67', 'Att68', 'Att69', 'Att70', 'Att71', 'Att72', 'Att73', 'Att74', 'Att75', 'Att76', 'Att77', 'Att78', 'Att79', 'Att80', 'Att81', 'Att82', 'Att83', 'Att84', 'Att85', 'Att86', 'Att87', 'Att88', 'Att89', 'Att90', 'Att91', 'Att92', 'Att93', 'Att94', 'Att95', 'Att96', 'Att97', 'Att98', 'Att99', 'Att100', 'Att101', 'Att102', 'Att103'],[]],\
                       'diabete':[['age','time_in_hospital','num_lab_procedures','num_procedures','num_medications','number_outpatient','number_emergency','number_inpatient','number_diagnoses'],['race','gender','admission_type_id','discharge_disposition_id','admission_source_id','max_glu_serum','A1Cresult','diabetesMed','metformin','repaglinide','nateglinide','chlorpropamide','glimepiride','acetohexamide','glipizide','glyburide','tolbutamide','pioglitazone','rosiglitazone','acarbose','miglitol','troglitazone','tolazamide','examide','citoglipton','insulin','glyburide-metformin','glipizide-metformin','glimepiride-pioglitazone','metformin-rosiglitazone','metformin-pioglitazone','change','readmitted','diag_1','diag_2','diag_3']],\
                       'woman':[['n_0067','n_0078','n_0108','n_0109','o_0176','o_0264'],['release','n_0047','n_0050','n_0052','n_0061','n_0075','n_0091','c_0466','c_0500','c_0638','c_0699','c_0738','c_0761','c_0770','c_0838','c_0870','c_0980','c_1145','c_1158','c_1189','c_1223','c_1227','c_1244','c_1259']]}

# Yeast Dataset

In [ ]:
df_yeast = pd.DataFrame(arff.loadarff('../dataset/dataset_raw/yeast.arff')[0])

for col in df_yeast.columns[-14:]:
    df_yeast[col] = df_yeast[col].apply(pd.to_numeric)

In [ ]:
df_yeast.head()

In [ ]:
cols_Y = [col for col in df_yeast.columns if col.startswith('Class')]
cols_X = [col for col in df_yeast.columns if col not in cols_Y]

X = df_yeast[cols_X].values
y = df_yeast[cols_Y].values

In [ ]:
X_bb, X_2e, y_bb, y_2e = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
df_bb = pd.DataFrame(data=np.concatenate((X_bb, y_bb), axis=1), columns=df_yeast.columns)
df_2e = pd.DataFrame(data=np.concatenate((X_2e, y_2e), axis=1), columns=df_yeast.columns)

In [ ]:
df_bb.to_csv('../dataset/yeast_bb.csv', sep=',', index=False)
df_2e.to_csv('../dataset/yeast_2e.csv', sep=',', index=False)

# Diabete Dataset

Given the description of the variables in https://www.hindawi.com/journals/bmri/2014/781670/tab1/, we decided to drop those with high percentages of missing values and those who were not important for the classification (patient id or encounter id), the target variables are the three types of diagnosis (primary, secondary and additional secondary diagnosis)

In [ ]:
df_diabete = pd.read_csv('../dataset/dataset_raw/diabetic_data.csv', sep=',', skipinitialspace=True, na_values='?')

In [ ]:
mv = df_diabete.isnull().sum(axis=0)
for k, v in zip(mv.index, mv.values):
    if v > 0:
        print(k, v, '%.2f' % (v/len(df_diabete)))

In [ ]:
df_diabete = df_diabete.drop(['encounter_id','patient_nbr','weight','payer_code','medical_specialty'],1)

In [ ]:
len(df_diabete),len(df_diabete.dropna())

In [ ]:
df_diabete = df_diabete.dropna().reset_index().drop('index',1)

In [ ]:
df_diabete = df_diabete.sample(n=14644,random_state=0).reset_index().drop('index',1)

In [ ]:
diag_1_numerosity = df_diabete.groupby('diag_1').size()
frequent_diag1 = diag_1_numerosity[diag_1_numerosity.values>np.percentile(diag_1_numerosity.values,75)].index.values

diag_2_numerosity = df_diabete.groupby('diag_2').size()
frequent_diag2 = diag_2_numerosity[diag_2_numerosity.values>np.percentile(diag_2_numerosity.values,75)].index.values

diag_3_numerosity = df_diabete.groupby('diag_3').size()
frequent_diag3 = diag_3_numerosity[diag_3_numerosity.values>np.percentile(diag_3_numerosity.values,75)].index.values

In [ ]:
df_diabete=df_diabete[(df_diabete.diag_1.isin(frequent_diag1))&(df_diabete.diag_2.isin(frequent_diag2))&(df_diabete.diag_3.isin(frequent_diag3))].reset_index().drop('index',1)

In [ ]:
df_diabete.shape

In [ ]:
df_diabete[['diag_1','diag_2','diag_3']].head()

In [ ]:
#age diventa la metà del range (var continua)
df_diabete['age']=df_diabete.age.apply(lambda x: float(x.strip('[').strip(')').split('-')[1])-5)

In [ ]:
len(columns_type_dataset['diabete'][1]+columns_type_dataset['diabete'][0]),len(df_diabete.columns)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(categories='auto')
encoded_dataframes = []
categorical_variables_diabetes = []

for col in columns_type_dataset['diabete'][1]:
    enc.fit(df_diabete[col].values.reshape(-1,1))
    categories_names = enc.categories_
    columns_names = [col+'='+str(name) for name in categories_names[0]]
    values_encoded = enc.transform(df_diabete[col].values.reshape(-1, 1)).toarray()
    encoded_dataframes.append(pd.DataFrame(values_encoded, columns=columns_names))
    categorical_variables_diabetes.append(columns_names)

categorical_variables_diabetes_names = [item for sublist in categorical_variables_diabetes for item in sublist]


diabete_encoded = pd.concat([pd.concat(encoded_dataframes,1),df_diabete[columns_type_dataset['diabete'][0]]],1)
diabete_encoded.head()

In [ ]:
target_cols = [col for col in diabete_encoded.columns if 'diag_' in col]

s = diabete_encoded[target_cols].loc[0]
s.iloc[s.nonzero()[0]]

In [ ]:
columns_type_dataset['diabete'][1] = [i for i in categorical_variables_diabetes_names if 'diag_' not in i]

In [ ]:
cols_Y = [col for col in diabete_encoded.columns if col.startswith('diag_')]
cols_X = [col for col in diabete_encoded.columns if col not in cols_Y]

X = diabete_encoded[cols_X].values
y = diabete_encoded[cols_Y].values

In [ ]:
diabete_encoded[cols_Y].values

In [ ]:
print('number of columns: %d \nsum of cols X + cols Y: %d \nnumber of features: %d \nnumber of target cols: %d' %(len(diabete_encoded.columns),len(cols_X)+len(cols_Y),len(cols_X),len(cols_Y)))

In [ ]:
X_bb, X_2e, y_bb, y_2e = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
print(X_bb.shape)
print(y_bb.shape)

In [ ]:
df_bb = pd.concat([pd.DataFrame(data=X_bb,columns=cols_X),pd.DataFrame(data=y_bb,columns=cols_Y)],1)
df_2e = pd.concat([pd.DataFrame(data=X_2e,columns=cols_X),pd.DataFrame(data=y_2e,columns=cols_Y)],1)

In [ ]:
df_bb.to_csv('../dataset/diabete_bb.csv', sep=',', index=False)
df_2e.to_csv('../dataset/diabete_2e.csv', sep=',', index=False)

# Woman Health Care

In [ ]:
woman = pd.read_csv('../dataset/dataset_raw/women_health_care.csv', sep=',')

In [ ]:
woman.head()

In [ ]:
len(woman)

In [ ]:
mv = woman.isnull().sum(axis=0)

columns2drop = list()
for k, v in zip(mv.index, mv.values):
    if v != 0.0:
        columns2drop.append(k)

In [ ]:
woman.drop(columns2drop, axis=1, inplace=True)

In [ ]:
enc = OneHotEncoder(categories='auto')
encoded_dataframes = []
categorical_variables_woman = []

for col in columns_type_dataset['woman'][1]:
    enc.fit(woman[col].values.reshape(-1,1))
    categories_names = enc.categories_
    columns_names = [col+str(name) for name in categories_names[0]]
    values_encoded = enc.transform(woman[col].values.reshape(-1, 1)).toarray()
    encoded_dataframes.append(pd.DataFrame(values_encoded, columns=columns_names))
    categorical_variables_woman.append(columns_names)

categorical_variables_woman_names = [item for sublist in categorical_variables_woman for item in sublist]

woman_encoded = pd.concat([pd.concat(encoded_dataframes,1),woman.drop(columns_type_dataset['woman'][1],1)],1)
woman_encoded.head()

In [ ]:
columns_type_dataset['woman'][1] = categorical_variables_woman_names

In [ ]:
df_hc_label = pd.read_csv('../dataset/dataset_raw/women_health_care_labels.csv', sep=',')

In [ ]:
df_hc_label.head()

In [ ]:
len(df_hc_label)

In [ ]:
df_hc = woman_encoded.set_index('id').join(df_hc_label.set_index('id'), how='inner').reset_index().drop('id',1)

In [ ]:
cols_Y = [col for col in df_hc.columns if col.startswith('service')]
cols_X = [col for col in df_hc.columns if col not in cols_Y]

X = df_hc[cols_X].values
y = df_hc[cols_Y].values

In [ ]:
X_bb, X_2e, y_bb, y_2e = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
df_bb = pd.DataFrame(data=np.concatenate((X_bb, y_bb), axis=1), columns=df_hc.columns)
df_2e = pd.DataFrame(data=np.concatenate((X_2e, y_2e), axis=1), columns=df_hc.columns)

In [ ]:
df_bb.to_csv('../dataset/woman_bb.csv', sep=',', index=False)
df_2e.to_csv('../dataset/woman_2e.csv', sep=',', index=False)

#### Save dictionary of varibles names divided in categorical and continuous ones for later use

In [ ]:
#columns_type_dataset.pop('diabete')

In [ ]:
with open('../dataset/dict_names.pickle', 'wb') as handle:
    pickle.dump(columns_type_dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)